# Armed Conflict Location Event Data

"The Armed Conflict Location & Event Data Project (ACLED) is a disaggregated data collection, analysis, and crisis mapping project. ACLED collects the dates, actors, locations, fatalities, and types of all reported political violence and protest events across Africa, the Middle East, Latin America & the Caribbean, East Asia, South Asia, Southeast Asia, Central Asia & the Caucasus, Europe, and the United States of America. The ACLED team conducts analysis to describe, explore, and test conflict scenarios, and makes both data and analysis open for free use by the public."

"ACLED is a registered non-profit organization with 501(c)(3) status in the United States."

Copied from [about ACLED](https://acleddata.com/about-acled/)

[ACLED website](https://acleddata.com/#/dashboard)

# 0. Import Python Dependencies


In [1]:
#@title
#import libraries
!pip install pyproj
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, Layout
import json
from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Legend, ColumnDataSource,HoverTool
from bokeh.palettes import Colorblind8, Set1_9,Paired12, Category20_20, Turbo256
palette = Colorblind8+Set1_9+Paired12+Category20_20+Turbo256
from bokeh.transform import factor_cmap
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models.formatters import DatetimeTickFormatter
from pyproj import Transformer
from google.colab import files
import io
tile_provider = get_provider('STAMEN_TERRAIN')
output_notebook()


     |████████████████████████████████| 6.5MB 4.0MB/s 


# 1. Upload File

Enter name of the ACLED data file you are uploading with the .csv extension (i.e. "file.csv") and select from your machine to upload.

In [2]:
#@title
upload =widgets.Text(placeholder = "Type Filename.csv",
        description="File", 
        disabled = False)


actor_choice = []
def upload_file(filename):
    return filename

actor_selection = interact(upload_file, filename=upload)



interactive(children=(Text(value='', description='File', placeholder='Type Filename.csv'), Output()), _dom_cla…

# 2. Select the file to upload

You can see the file by clicking on the folder icon in the upper left. 

In [3]:
#@title
data_to_load = files.upload()
df = pd.read_csv(io.BytesIO(data_to_load[upload.value]))
df

Saving ACLED_Philippines.csv to ACLED_Philippines.csv


,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7665243,608,PHL11698,11698,2/5/2021,2021,1,Protests,Peaceful protest,Protesters (Philippines),KADAMAY: Alliance of Urban Poor Organizations,6,NaN,NaN,0,60,Southeast Asia,Philippines,National Capital Region,Metro Manila,Manila,Manila,14.6042,120.9822,1,Bulatlat,National,"On 5 February 2021, demonstrators from urban p...",0,1612888443,PHL
1,7665227,608,PHL11721,11721,2/5/2021,2021,1,Battles,Armed clash,NPA: New People's Army,NaN,2,Military Forces of the Philippines (2016-),NaN,1,12,Southeast Asia,Philippines,Calabarzon,Quezon,Catanauan,San Roque,13.6429,122.3577,1,Philippine Daily Inquirer; Abante,National,"On 5 February 2021, 2 government soldiers were...",2,1612888443,PHL
2,7665226,608,PHL11720,11720,2/5/2021,2021,1,Battles,Armed clash,NPA: New People's Army,NaN,2,Military Forces of the Philippines (2016-),NaN,1,12,Southeast Asia,Philippines,Calabarzon,Quezon,Buenavista,Masaya,13.6784,122.4018,1,Philippine Information Agency; Philippine Dail...,National,"On 5 February 2021, a suspected NPA militant w...",1,1612888443,PHL
3,7665308,608,PHL11719,11719,2/4/2021,2021,2,Violence against civilians,Attack,Unidentified Armed Group (Philippines),NaN,3,Civilians (Philippines),KMP: Peasant Movement of the Philippines; Farm...,7,37,Southeast Asia,Philippines,Calabarzon,Laguna,Santa Cruz,Santa Cruz,14.2814,121.4161,2,Manila Bulletin,National,Around 4 February 2021 (between 3 - 5 February...,1,1612888443,PHL
4,7665211,608,PHL11696,11696,2/4/2021,2021,1,Violence against civilians,Attack,Police Forces of the Philippines (2016-),NaN,1,Civilians (Philippines),BIFF: Bangsamoro Islamic Freedom Fighters; Pri...,7,17,Southeast Asia,Philippines,Bangsamoro Autonomous Region in Muslim Mindanao,Maguindanao,Rajah Buayan,Rajah Buayan,6.9342,124.5559,1,Manila Bulletin; Sunstar (Philippines); Philip...,National,"On 4 February 2021, 2 suspected BIFF militants...",2,1612888443,PHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11533,7089691,608,PHL5,5,1/4/2016,2016,2,Violence against civilians,Attack,Unidentified Armed Group (Philippines),NaN,3,Civilians (Philippines),LP: Liberal Party,7,37,Southeast Asia,Philippines,Caraga,Surigao del Norte,Bacuag,Bacuag,9.6092,125.6397,2,Philippine Star,National,A local leader of the Liberal Party (LP) in Su...,1,1589298675,PHL
11534,5615014,608,PHL5635,5635,1/4/2016,2016,1,Protests,Peaceful protest,Protesters (Philippines),NaN,6,NaN,NaN,0,60,Southeast Asia,Philippines,National Capital Region,Metro Manila,Manila,Manila,14.6042,120.9822,1,Bulatlat,National,Commuters staged a protest on 4 Jan 2016 at th...,0,1563887830,PHL
11535,7088007,608,PHL3,3,1/2/2016,2016,1,Battles,Armed clash,Unidentified Armed Group (Philippines),NaN,3,Unidentified Armed Group (Philippines),NaN,3,33,Southeast Asia,Philippines,Bangsamoro Autonomous Region in Muslim Mindanao,Sulu,Pata,Pata,5.8148,121.1639,2,Philippine Daily Inquirer,National,Armed followers of Pata Mayor Anton Burahan an...,0,1589298664,PHL
11536,7088006,608,PHL2,2,1/2/2016,2016,1,Battles,Armed clash,Unidentified Armed Group (Philippines),NaN,3,Military Forces of the Philippines (2010-2016),NaN,1,13,Southeast Asia,Philippines,Bangsamoro Autonomous Region in Muslim Mindanao,Maguindanao,Datu Saudi-Ampatuan,Pagatin,6.9730,124.4752,1,Philippine Daily Inquirer,National,Unidentified motorcycle-riding man shot dead a...,1,1589298664,PHL


# 3. If desired, select a category

The categories are based on the columns in the dataframe/table and are the features captured by ACLED. 



In [41]:
#@title
#Now I want to put them together into one giant result
sub_choices = sorted(list(df.columns))
sub_choices.insert(0, "all")
type_select = widgets.Dropdown(options = sub_choices,
    description='Sub Category',
    disabled=False)

def update(sub):
    return sub

res_selection = interact(update, sub= type_select)


interactive(children=(Dropdown(description='Sub Category', options=('all', 'actor1', 'actor2', 'admin1', 'admi…

# 4. If desired, select a subcategory. 



In [42]:
#@title
#Now I want to put them together into one giant result

if type_select.value == "all":
  cat_list = "all"
  print("This step is not necessary, you have selected all.")
else: 
  cat_list = sorted(list(df[type_select.value].unique()))
  cat_list.insert(0, "all")

  sub_select = widgets.SelectMultiple(options = cat_list,
                                      layout=Layout(width="50%", height="180px"),
                                      description='Category Select',
                                      disabled=False)

  def update(cat):
      return cat

  res_selection = interact(update, cat= sub_select)


interactive(children=(SelectMultiple(description='Category Select', layout=Layout(height='180px', width='50%')…

# 5. Map the events - Pick Map By Category

In [43]:
#@title
#convert lat/long to web mercator

df2 =pd.DataFrame()

if sub_select.value[0]=="all":
  selected = cat_list[1:]
else: 
  selected = sub_select.value


for val in selected: 
  df2 =pd.concat([df2,df[df.loc[:,type_select.value]==val]], ignore_index=False)


transformer = Transformer.from_crs('epsg:4326','epsg:3857')

def convert_loc(lat,long):
  point = list(transformer.transform(lat,long))
  return point

points = df2.apply(lambda row:convert_loc(row["latitude"], row["longitude"]), axis=1)
df2['web_lat'] = points.apply(lambda row: row[0])
df2["web_lon"] = points.apply(lambda row: row[1])

#get a bounding box
max_lat = df2.web_lat.max() 
min_lat = df2.web_lat.min()
max_lon = df2.web_lon.max()
min_lon = df2.web_lon.min()

map_by = sorted(list(df2.columns))
map_select = widgets.Dropdown(options = map_by,
    description='Map By',
    disabled=False)

def update(map):
    return map

map_selection = interact(update, map= map_select)


interactive(children=(Dropdown(description='Map By', options=('actor1', 'actor2', 'admin1', 'admin2', 'admin3'…

# 6. Build the Map

In [44]:
#@title
color = len(df[map_select.value].unique())
types = list(df[map_select.value].unique())
#Violates DRY should be a function
if type(types[0])!= str: 
  types = sorted([str(i) for i in types])

category = list(df2[map_select.value])
if type(category[0])!= str: 
  category = sorted([str(i) for i in category])
  
data2 = {"lats" : list(df2["web_lat"]), "lons": list(df2["web_lon"]), "category":category}

plot_points = ColumnDataSource(data=data2)

p = figure(x_range=(min_lat, max_lat),y_range=(min_lon, max_lon),x_axis_type="mercator", y_axis_type="mercator", toolbar_location = "above",
           height=600, width = 1200)
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
p.add_layout(Legend(), 'right')
#See Palette on import
p.scatter(x = "lats", y ="lons", source=plot_points, legend_group="category", color = factor_cmap("category",palette,types))

show(p)

# 7. Plot the Map By and Month

Creates a Time Series Plot of Map By selection and Month

First it is necessary to conslidate the data for plotting.

In [45]:
#@title
def fix_date(date):
  #print(date)
  date =date.split("/")
  return date[0]+"/"+date[2]

df2["short_event"] = df2.apply(lambda row: fix_date(row["event_date"]), axis=1)
actors = list(df2['actor1'].unique())

df2 = df2.groupby([map_select.value,"short_event"], as_index=False).agg({"count"})

df2

data_id   iso  ... web_lat web_lon
                                     count count  ...   count   count
actor1                 short_event                ...                
ASG: Abu Sayyaf        1/2016           13    13  ...      13      13
                       1/2017           10    10  ...      10      10
                       1/2018            4     4  ...       4       4
                       1/2019            6     6  ...       6       6
                       1/2020            4     4  ...       4       4
...                                    ...   ...  ...     ...     ...
NPA: New People's Army 8/2020           27    27  ...      27      27
                       9/2017           35    35  ...      35      35
                       9/2018           18    18  ...      18      18
                       9/2019           25    25  ...      25      25
                       9/2020           29    29  ...      29      29

[143 rows x 32 columns]

# 8. Plot the results

In [46]:
#@title
event_dict={}
for idx in range(len(actors)):
  event_dict[actors[idx]]={"num":[], "month":[], "color": palette[idx]}


for row, event in df2.iterrows(): 
    date = dt.strptime(row[1], '%m/%Y')
    event_dict[row[0]]['month'].append(date)
    event_dict[row[0]]['num'].append(event["event_date"]["count"])
    
for k,v in event_dict.items(): 
  l1= v["month"]
  l2= v["num"]
  event_dict[k]["month"], event_dict[k]["num"]=(list(t) for t in zip(*sorted(zip(l1,l2))))    

 
p = figure(plot_width=1200, plot_height=500, x_axis_type="datetime")
p.add_layout(Legend(), 'right')
p.legend.click_policy="hide"
p.xaxis[0].formatter = DatetimeTickFormatter(months=["%b %Y"], days = ["%m/%Y"])

for k,v in event_dict.items(): 
  p.line(v["month"], v["num"], line_width=2.0, color=v['color'], legend_label=k)

show(p)